In [1]:
# 1. Import dan Load CSV
import pandas as pd
import plotly.express as px
import re

df = pd.read_csv("steam_games.csv")
df = df[["popular_tags", "release_date", "all_reviews"]].dropna()
df.head()


,popular_tags,release_date,all_reviews
0,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","May 12, 2016","Very Positive,(42,550),- 92% of the 42,550 use..."
1,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Dec 21, 2017","Mixed,(836,608),- 49% of the 836,608 user revi..."
2,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Apr 24, 2018","Mostly Positive,(7,030),- 71% of the 7,030 use..."
3,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Dec 13, 2018","Mixed,(167,115),- 61% of the 167,115 user revi..."
4,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","May 6, 2003","Mostly Positive,(11,481),- 74% of the 11,481 u..."


In [2]:
# 2. Bersihkan release_year
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df = df.dropna(subset=["release_year"])
df["release_year"] = df["release_year"].astype(int)
df = df[df["release_year"].between(2000, 2025)]


In [3]:
# 3. Pecah genre jadi 1 baris per genre
df["genre_list"] = df["popular_tags"].str.split(",")
df = df.explode("genre_list")
df["genre_list"] = df["genre_list"].str.strip()
df = df[df["genre_list"] != ""]


In [4]:
# 4. Ekstrak rating % terakhir dari all_reviews
def extract_rating(txt):
    match = re.findall(r"(\d{1,3})%", str(txt))
    return int(match[-1]) if match else None

df["rating_percent"] = df["all_reviews"].apply(extract_rating)
df = df[df["rating_percent"].between(10, 100)]


In [5]:
# 5. PIE CHART: Top 15 Genre (Min 100 Game)
top_genre = (
    df.groupby("genre_list")
    .size()
    .reset_index(name="count")
    .query("count >= 100")
    .sort_values("count", ascending=False)
    .head(15)
)

pie_fig = px.pie(top_genre, names="genre_list", values="count",
                 title="Top 15 Genre (Min 100 Game)")
pie_fig.update_traces(textinfo="percent+label", pull=[0.05]*len(top_genre))
pie_fig.show()

In [6]:
# 6. BAR CHART: Genre populer tahun tertentu
year = 2018
df_year = df[df["release_year"] == year]
bar_df = (
    df_year.groupby("genre_list")
    .size()
    .reset_index(name="count")
    .sort_values("count", ascending=False)
    .head(15)
)

bar_fig = px.bar(bar_df, x="genre_list", y="count", title=f"Top Genre Tahun {year}")
bar_fig.update_layout(xaxis_tickangle=-45)
bar_fig.show()


In [7]:
# 7 Export Bar Chart untuk semua tahun (2000–2024) ke templates/
import plotly.io as pio
import os

os.makedirs("templates", exist_ok=True)

for y in range(2000, 2019):
    df_y = df[df["release_year"] == y]
    top = (
        df_y.groupby("genre_list")
        .size()
        .reset_index(name="count")
        .sort_values("count", ascending=False)
        .head(15)
    )
    fig = px.bar(top, x="genre_list", y="count", title=f"Top Genre Tahun {y}")
    fig.update_layout(xaxis_tickangle=-45)
    pio.write_html(fig, f"templates/bar_chart_{y}.html", include_plotlyjs=False, full_html=False)


In [8]:
#  8. LINE CHART: Tren genre per tahun
line_df = (
    df.groupby(["release_year", "genre_list"])
    .size()
    .reset_index(name="count")
)

pivot = line_df.pivot(index="release_year", columns="genre_list", values="count").fillna(0)
pivot = pivot[[c for c in pivot.columns if pivot[c].sum() >= 100]]
df_long = pivot.reset_index().melt(id_vars="release_year", var_name="Genre", value_name="Jumlah Game")

line_fig = px.line(df_long, x="release_year", y="Jumlah Game", color="Genre",
                   title="Tren Jumlah Game per Genre")
line_fig.update_traces(mode="lines+markers")
line_fig.show()

In [9]:
# 9. CHART PERTUMBUHAN GAME PER TAHUN
growth_df = (
    df.groupby("release_year")
    .size()
    .reset_index(name="jumlah_game")
)

growth_fig = px.line(
    growth_df,
    x="release_year",
    y="jumlah_game",
    markers=True,
    title="Pertumbuhan Jumlah Game Steam per Tahun"
)
growth_fig.update_traces(line=dict(width=3), marker=dict(size=6))
growth_fig.show()


In [10]:
# 10. SCATTER CHART: Harga vs Diskon
df_diskon = pd.read_csv("steam_games.csv")
df_diskon = df_diskon[["name", "original_price", "discount_price"]].dropna()

# Ubah ke numeric (hilangkan simbol $ atau teks)
def parse_price(x):
    try:
        return float(str(x).replace("$", "").replace(",", "").strip())
    except:
        return None

df_diskon["original_price"] = df_diskon["original_price"].apply(parse_price)
df_diskon["discount_price"] = df_diskon["discount_price"].apply(parse_price)
df_diskon = df_diskon.dropna()

# Buat kolom diskon %
df_diskon["discount_percent"] = (
    (df_diskon["original_price"] - df_diskon["discount_price"]) / df_diskon["original_price"] * 100
).round(1)

# Filter diskon masuk akal
df_diskon = df_diskon[df_diskon["discount_percent"].between(0, 100)]

# Scatter Chart
scatter_fig = px.scatter(
    df_diskon,
    x="original_price",
    y="discount_percent",
    hover_name="name",
    title="Scatter: Harga Asli vs Diskon (%)",
    labels={"original_price": "Harga Asli ($)", "discount_percent": "Diskon (%)"},
    opacity=0.6
)
scatter_fig.update_traces(marker=dict(size=8, line=dict(width=1, color="DarkSlateGrey")))
scatter_fig.show()


In [11]:
# 10. Export ke HTML fragment
import plotly.io as pio

pio.write_html(pie_fig, "pie_chart.html", include_plotlyjs='cdn', full_html=False)
pio.write_html(bar_fig, "bar_chart.html", include_plotlyjs='cdn', full_html=False)
pio.write_html(line_fig, "line_chart.html", include_plotlyjs='cdn', full_html=False)
pio.write_html(growth_fig, "growth_chart.html", include_plotlyjs='cdn', full_html=False)
pio.write_html(scatter_fig, "scatter_chart.html", include_plotlyjs='cdn', full_html=False)




In [14]:
# 11. AI AUTO-INSIGHT GENERATOR
def generate_growth_insight(growth_df):
    fastest_year = growth_df.sort_values("jumlah_game", ascending=False).iloc[0]["release_year"]
    slowest_year = growth_df.sort_values("jumlah_game").iloc[0]["release_year"]
    latest_year = growth_df["release_year"].max()
    latest_games = growth_df[growth_df["release_year"] == latest_year]["jumlah_game"].values[0]

    insight = f"""
    Tahun dengan rilis game terbanyak adalah {int(fastest_year)}.
    Tahun dengan rilis game paling sedikit adalah {int(slowest_year)}.
    Pada tahun {int(latest_year)}, Steam menambahkan {int(latest_games)} game.
    """
    return insight.strip()

# Generate insight dan simpan ke file
growth_insight = generate_growth_insight(growth_df)

# Simpan ke file teks (biar Flask bisa baca)
with open("templates/growth_insight.txt", "w") as f:
    f.write(growth_insight)


In [13]:
# 12. AI GAME RECOMMENDATION (fungsi)
def recommend_games(df, genre=None, year=None, min_rating=70):
    df_rec = df.copy()
    if genre:
        df_rec = df_rec[df_rec["genre_list"].str.contains(genre, case=False, na=False)]
    if year:
        df_rec = df_rec[df_rec["release_year"] == year]
    df_rec = df_rec[df_rec["rating_percent"] >= min_rating]
    
    recs = (
        df_rec[["popular_tags", "release_year", "rating_percent"]]
        .dropna()
        .sort_values("rating_percent", ascending=False)
        .head(5)
    )
    return recs
